In [3]:
import json
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [5]:
# Set dataset path
dataset_path = 'dog-breeds'

In [6]:
# Image data generator for training and validation with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
# Increase image size for better feature extraction
img_size = (224, 224)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 435 images belonging to 8 classes.
Found 106 images belonging to 8 classes.


In [9]:
# Use transfer learning with MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

In [10]:
# Compile the model with a lower initial learning rate
initial_learning_rate = 1e-4
model.compile(optimizer=Adam(learning_rate=initial_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    epochs=15,  # Changed from 50 to 15
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/15
14/14 [==============================] - 25s 2s/step - loss: 2.2973 - accuracy: 0.1977 - val_loss: 1.5690 - val_accuracy: 0.6604 - lr: 1.0000e-04
Epoch 2/15
14/14 [==============================] - 19s 1s/step - loss: 1.5517 - accuracy: 0.4690 - val_loss: 1.1514 - val_accuracy: 0.8962 - lr: 1.0000e-04
Epoch 3/15
14/14 [==============================] - 18s 1s/step - loss: 1.2239 - accuracy: 0.6368 - val_loss: 0.7994 - val_accuracy: 0.9528 - lr: 1.0000e-04
Epoch 4/15
14/14 [==============================] - 18s 1s/step - loss: 0.9093 - accuracy: 0.7862 - val_loss: 0.5662 - val_accuracy: 0.9340 - lr: 1.0000e-04
Epoch 5/15
14/14 [==============================] - 20s 1s/step - loss: 0.6260 - accuracy: 0.8575 - val_loss: 0.3970 - val_accuracy: 0.9623 - lr: 1.0000e-04
Epoch 6/15
14/14 [==============================] - 17s 1s/step - loss: 0.5275 - accuracy: 0.8828 - val_loss: 0.2973 - val_accuracy: 0.9434 - lr: 1.0000e-04
Epoch 7/15
14/14 [==============================] - 17s 1s

In [13]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

4/4 [==============================] - 3s 676ms/step - loss: 0.4374 - accuracy: 0.8962
Validation Accuracy: 89.62%


In [15]:
# Save the model in the recommended .keras format
model.save('dog_breed_classifier.keras')
print("Model saved as 'dog_breed_classifier.keras'")

Model saved as 'dog_breed_classifier.keras'


In [16]:
# Save the class labels as a JSON file
class_labels = train_generator.class_indices
with open('class_labels.json', 'w') as f:
    json.dump(class_labels, f)
print("Class labels saved as 'class_labels.json'")

Class labels saved as 'class_labels.json'
